# Init

In [9]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import plotly.express as px
import json

In [2]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

In [3]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


In [21]:
data_project_score = pd.read_csv("hard-database/data_project_scoring.csv").rename(columns={"scoing_made":"scoring_made"})
data_project_score['project_len_char'] = data_project_score['project'].apply(len)

In [105]:
data_project_score_select = data_project_score[data_project_score['project_score']>20]
data_project_score_select.head(3)

,Unnamed: 0,slug,url,name,name_initial,short_title,financers,financers_full,instructors,instructors_full,...,description_md,eligibility_md,token_numb_description_md,token_numb_description,token_numb_eligibility_md,token_numb_eligibility,project,project_score,scoring_made,project_len_char
146,143318,06ae-maitriser-le-volet-nuisances-sonores-dans...,/aides/06ae-maitriser-le-volet-nuisances-sonor...,Maîtriser le volet « nuisances sonores » dans ...,NaN,NaN,['Cerema'],"[{'id': 58, 'name': 'Cerema', 'logo': 'https:/...",[],[],...,\n**Description :**\n\n\n\n\n Le bruit routier...,\n**Public :** \n\n\n\n\n La formation s'adres...,718,920,150,182,Revitalisation d'une zone humide,23,5,32
401,162653,aider-les-commerces-a-se-moderniser,/aides/aider-les-commerces-a-se-moderniser/,Moderniser les commerces,Aide aux Commerces des Territoires (ACTe),NaN,['Communauté de communes des Falaises du Talou'],"[{'id': 2475, 'name': 'Communauté de communes ...",[],[],...,**Objectifs**\n\nLa Communauté de Communes\r\n...,Les entreprises doivent répondre\r\naux critèr...,1913,2568,670,1170,Revitalisation d'une zone humide,23,5,32
525,58670,eea7-appel-a-projets-0-phyto,/aides/eea7-appel-a-projets-0-phyto/,Réduire l'usage des produits phytosanitaires,NaN,NaN,"[""Conseil régional d'Occitanie""]","[{'id': 91, 'name': ""Conseil régional d'Occita...",[],[],...,\n Contexte et objectifs\n\n\n\n\n La région O...,Consultez la page de l'aide pour obtenir des d...,790,957,44,44,Revitalisation d'une zone humide,23,5,32


In [135]:
def gen_prompt_question_mono_sub(sub_description:str, sub_eligibility:str, project_description:str, num_question:int=3)->(str,str):
    prompt_system = f"""Tu es une IA bienveillante qui aide l'utilisateur à préciser les éléments de son projet en lien avec une aide ou une subvention.

    Ton rôle est de poser des questions simples et encourageantes, permettant à l'utilisateur d'explorer son projet plus en détail et en lien avec les critères de la subvention.

    Tu répondras exclusivement au format JSON, en respectant strictement le format demandé."""
    
    prompt_user = f""" **Instruction**
    
    Pose exactement {num_question} questions bienveillantes et constructives en lien avec l’aide/subvention et le projet. Les questions doivent inciter l’utilisateur à clarifier des détails pratiques ou concrets en lien avec le programme, sans se perdre dans des informations superflues.

    Tu répondras au format JSON suivant : {{"Q1": str, "Q2": str, "Q3": str}}

    Utilise les informations suivantes pour déterminer les questions :
    
    - **Description de l'aide/subvention :**
    {sub_description}

    - **Description de l'aide/subvention :**
    {sub_eligibility}
    
    - **Description du projet :**
    {project_description}
    """

    return prompt_system, prompt_user

def gen_prompt_question_multi_sub(sub_description_list:list[str], project_description:str, num_question=3)->(str,str):
    prompt_system = f"""Tu es une IA bienveillante qui aide l'utilisateur à préciser les éléments de son projet en lien avec des aides ou des subventions.

    Ton rôle est de poser des questions simples et encourageantes, permettant à l'utilisateur d'explorer son projet plus en détail et en lien avec les critères des subventions.

    Tu répondras exclusivement au format JSON, en respectant strictement le format demandé."""
    

    sub_description_list_text = "".join([f"**\n-Description aide {i} :**\n{sub_description_list[i]}" for i in range(len(sub_description_list))])

    prompt_user = f""" **Instruction**
    
    Pose exactement {num_question} questions bienveillantes et constructives en lien avec la list de subvention et le projet. Les questions doivent inciter l’utilisateur à clarifier des détails pratiques ou concrets en lien avec le programme, sans se perdre dans des informations superflues.

    Tu répondras au format JSON suivant : {{"Q1": str, "Q2": str, "Q3": str}}

    Utilise les informations suivantes pour déterminer les questions :
    {sub_description_list_text}

    - **Description du projet :**
    {project_description}
    """

    return prompt_system, prompt_user

In [57]:
row = data_project_score_select[['description_md','eligibility_md','project']].iloc[0]

sub_description,sub_eligibility,project_description = row['description_md'],row['eligibility_md'],row['project']

In [85]:
model = "qwen2.5:14b"
model_options = model_data[model]

request_options = {
    "num_ctx": 16384,
    "num_predict": 512
}

num_question = 3


prompt_system, prompt_user = gen_prompt_question_mono_sub(sub_description,sub_eligibility,project_description,num_question)

response = generate_ollama_request(
    prompt_system=prompt_system,
    response_format="json",
    prompt_user=prompt_user,
    ollama_api_url=ollama_api_url,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    model=model, 
    request_options= request_options,  # Default to None
    seed=0,
    )

In [72]:
response['response']

'{"Q1":"Pourquoi est-ce important de bien comprendre la réglementation relative au bruit routier ? ","" \n\n      :"En effet, le CNB et l\'ADEME ont publié en juillet 2021 une nouvelle évaluation du coût social des infrastructures de transport routière qui indique que les activités liées aux transports sont responsables d\'un budget important pour la France. Mais aussi qu\'en vous-même ? Quels impacts peut votre projet avoir sur l’environnement et vos voisins, si tu as un certain impact en lien avec le programme?"}\n\n \n\n\n\n\n\n     \n\n\n\n  \n\n\n\n\n   \n\n\n\n\n\n\n\n    \n    \n\n\n           \n\n         \n\n\n\n        \n\n\n\n\n\n\n      '

In [86]:
json_text = response['response']

# Remplacer les caractères HTML par une apostrophe
json_text = json_text.replace('&#39;', "'")

# Convertir la chaîne en dictionnaire Python
response_json = json.loads(json_text)

response_json

{'Q1': "Pourriez-vous préciser comment la formation sur l'étude acoustique s'appliquerait à votre projet de revitalisation d’une zone humide ?",
 'Q2': 'Quelles sont les réglementations spécifiques liées au bruit que vous devrez prendre en compte dans le cadre du développement durable et environnemental pour ce type de projets ? ',
 'Q3': "Avez-vous déjà des connaissances sur l'acoustique applicables aux infrastructures routières ou cette formation serait-elle plus adaptée comme point d’entrée initial?"}

In [93]:
data_project_score_select[data_project_score_select['project']=="Revitalisation d'une zone humide"].description_md.values[:3]

array(["\n**Description :**\n\n\n\n\n Le bruit routier est la première source de nuisance citée par les Français à leur domicile. Le Conseil National du Bruit (CNB) et l'Agence de la Transition écologique (ADEME) ont publié en juillet 2021 une nouvelle évaluation du coût social du bruit en France qui indique que le bruit des infrastructures de transport routières représente un coût de plus de 80 milliards d'euros par an.\n\n\n\n\n Le bruit émis par les infrastructures de transport routières s'inscrit dans un corpus réglementaire ancien, cependant l'Autorité environnementale (Ae) pointe régulièrement les lacunes dans les études acoustiques, qui peuvent impacter directement la qualité de vie des riverains de ces projets.\n\n\n\n\n Cette formation proposée en deux modules vous aidera à mieux appréhender les obligations des gestionnaires routiers, à conduire une étude d'impact acoustique et plus globalement à comprendre l'articulation des différents axes de la politique de lutte contre le 

In [108]:
project_list = data_project_score_select.project.unique()

In [124]:
number_of_sub = 3

data_list_out = []

for project_description in project_list:
    data_project_best = data_project_score_select[data_project_score_select['project']==project_description].sort_values("project_score")[-number_of_sub:]
    sub_description_list = data_project_best['description_md'].values
    sub1,sub2,sub3 = sub_description_list

    try:
        prompt_system, prompt_user = gen_prompt_question_multi_sub(sub_description_list,project_description,num_question)
        response = generate_ollama_request(
            prompt_system=prompt_system,
            response_format="json",
            prompt_user=prompt_user,
            ollama_api_url=ollama_api_url,
            bearer_token=ollama_bearer_token,
            model_options = model_options,  # Default to None
            model=model, 
            request_options= request_options,  # Default to None
            seed=0,
            )

        json_text = response['response']

        # Remplacer les caractères HTML par une apostrophe
        json_text = json_text.replace('&#39;', "'")

        # Convertir la chaîne en dictionnaire Python
        response_json = json.loads(json_text)

        q1 = response_json['Q1']
        q2 = response_json['Q2']
        q3 = response_json['Q3']

        data_list_out.append([project_description,sub1,sub2,sub3,q1,q2,q3])
        print('---------------')
        print(project_description)
    except Exception as error:
        print(error)

---------------
Revitalisation d'une zone humide
---------------
Entretient d'un vieux moulin
---------------
Réhabilitation d'une ancienne école en lieu dédié à la santé.
---------------
Voir fiche action PVD /ORT n°21 Après une première phase d’aménagement, la commune poursuit son action dans le cadre d’une 2ème phase d’aménagement. Cette prochaine étape concerne : - La mise en accessibilité du site aux personnes en situation de handicap et l’installation d’équipements en accès libre favorisant la pratique physique. - La protection de l’environnement par l’installation de sanitaires autonomes et la création d’un site O déchets - La création d'un parcours pédagogique
---------------
La commune dispose d'outils numériques qu'il est nécessaire d'optimiser et coordonner
---------------
Création d’un sentier thématique sur la forêt à Rieutord et sur le patrimoine à Usclades
---------------
Je souhaite refaire la voirie communal
---------------
Le projet consiste en l’aménagement d’un terr

In [130]:
data_question = pd.DataFrame(data_list_out,columns=['project','sub1','sub2','sub3','q1','q2','q3'])
data_question.to_csv('hard-database/data_question.csv')